In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
import os
from sklearn.preprocessing import StandardScaler, scale
import lightgbm as lgb

print(os.listdir("../input"))

['test_v2.csv', 'sampleSubmission.csv', 'train_v2.csv']


In [2]:
data = pd.read_csv('../input/train_v2.csv')
print ("Data is ready!")

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (135,204,274,417) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Data is ready!


- Taking top 90000 data for traing & testing perpose. 

In [3]:
data = data.head(90000)

In [4]:
print("Initial Dataset")
print(data.shape[0], " rows & ", data.shape[1], " columns(features).")

Initial Dataset
90000  rows &  771  columns(features).


If we observe the data:
- total 769 anonymous features.
- 2 labeled features (id & loss).
- loss is our target feature which contains integer values between(0-100) indecating percentage of loss(>0) if the loan is default otherwise 0.

In [5]:
data.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,...,f740,f741,f742,f743,f744,f745,f746,f747,f748,f749,f750,f751,f752,f753,f754,f755,f756,f757,f758,f759,f760,f761,f762,f763,f764,f765,f766,f767,f768,f769,f770,f771,f772,f773,f774,f775,f776,f777,f778,loss
0,1,126,10,0.686842,1100,3,13699,7201.0,4949.0,126.75,126.03,7,0.7607,0.7542,612922,0.7236,0.7236,0.5171,0.7236,0.8476,0.7876,1.097851e+09,89,66,998046.0,89.0,89.0,89.00,89.00,89.0,0.9271,0,0,0,5,0,0,0.73180,0.72124,0.749280,...,-4.40,6,9.76,-21.82,49.18,1.0435,-0.9066,1.5883,3.4154,2.8283,2.3451,0.030594,1.7418,1.5271,0.8474,0.4715,0.028362,3.1611,2.5162,2.0037,0.019636,4.4352,4.2676,-0.1524,1,-0.40,-0.560,-0.440,-0.6280,-3.14,5,2.14,-1.54,1.18,0.1833,0.7873,1,0,5,0
1,2,121,10,0.782776,1100,3,84645,240.0,1625.0,123.52,121.35,7,0.6555,0.6555,245815,0.6341,0.6341,0.3909,0.6667,0.6903,0.6903,8.449459e+08,78,50,754416.0,78.0,78.0,78.00,78.00,78.0,0.8478,0,0,0,6,0,0,0.65705,0.64069,0.626200,...,0.35,7,51.73,-72.44,798.74,2.7180,-0.8294,1.0740,2.9194,2.1180,1.5666,0.120442,1.1963,1.0322,0.4843,0.2389,0.130160,2.7659,1.9523,1.4059,0.115277,3.2763,2.7962,-0.3097,1,-0.17,-0.275,-0.203,-0.2300,-1.38,6,0.54,-0.24,0.13,0.1926,-0.6787,1,0,5,0
2,3,126,10,0.500080,1100,3,83607,1800.0,1527.0,127.76,126.49,7,0.7542,0.7542,1385872,0.7542,0.7542,0.5508,0.7542,0.8091,0.7807,1.308478e+09,89,54,1037651.0,89.0,89.0,100.43,94.37,89.0,0.9175,0,0,0,13,0,0,0.74710,0.75878,0.654685,...,-6.35,14,56.62,-79.48,621.97,1.9592,-0.3632,3.2969,7.2072,5.7062,4.5627,0.226336,3.3277,3.4166,1.8321,0.9979,0.103307,6.8623,5.2963,4.1282,0.219729,8.1381,7.3269,-0.1909,1,-0.58,-0.540,-0.572,-0.3985,-5.18,13,2.89,-1.73,1.04,0.2521,0.7258,1,0,5,0
3,4,134,10,0.439874,1100,3,82642,7542.0,1730.0,132.94,133.58,7,0.8017,0.7881,704687,0.7881,0.7881,0.5923,0.7881,0.8230,0.8158,1.472752e+09,93,55,1115721.0,93.0,93.0,114.63,102.92,93.0,0.9688,0,0,0,4,0,0,0.77405,0.78929,0.750675,...,3.75,5,36.31,18.15,429.81,2.5883,-0.8812,1.2950,2.5777,2.0830,1.6899,0.054630,1.3748,1.3421,0.7982,0.4810,0.081205,2.5571,2.0593,1.6653,0.056470,3.2516,3.0631,-0.1770,1,-0.75,-0.635,-0.745,-0.5100,-2.04,4,1.29,-0.89,0.66,0.2498,0.7119,1,0,5,0
4,5,109,9,0.502749,2900,4,79124,89.0,491.0,122.72,112.77,6,0.5263,0.5263,51985,0.5263,0.5263,0.3044,0.5405,0.5556,0.5455,1.442916e+09,60,21,536400.0,60.0,60.0,60.00,60.00,60.0,0.8451,0,0,0,26,0,0,0.60315,0.53367,0.696150,...,11.45,27,2206.14,-10559.05,708436.49,9.0293,-0.8760,6.3852,17.2599,14.2795,11.9179,0.085330,7.2175,6.2262,3.1446,1.6149,0.074286,15.9080,12.5688,9.9844,0.067540,17.5561,15.6079,-0.4444,1,-0.18,-0.280,-0.182,-0.4277,-11.12,26,6.11,-3.82,2.51,0.2282,-0.5399,0,0,5,0


- storing non default loan data in NoLoss dataframe.
- storing default loan data in Loss dataframe.

In [6]:
NoLoss = data.loc[data['loss']==0]
Loss = data.loc[data['loss']!=0]
print("Non Default loans in dataset:", NoLoss.shape[0])
print("Default loans in dataset:", Loss.shape[0])

Non Default loans in dataset: 81769
Default loans in dataset: 8231


In [7]:
Loss.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,...,f740,f741,f742,f743,f744,f745,f746,f747,f748,f749,f750,f751,f752,f753,f754,f755,f756,f757,f758,f759,f760,f761,f762,f763,f764,f765,f766,f767,f768,f769,f770,f771,f772,f773,f774,f775,f776,f777,f778,loss
7,8,128,9,0.385778,2900,4,79244,6597.0,3592.0,127.19,127.89,6,0.8596,0.7967,17175731,0.8033,0.7538,0.5868,0.8376,0.9159,0.8522,3.127331e+09,98,75,1219708.0,98.0,98.0,120.79,108.46,98.0,0.9899,0,0,0,17,0,0,0.72690,0.73842,0.739165,...,21.49,18,177.11,361.87,5280.10,2.9007,-0.7834,5.5179,11.5597,9.6605,8.1316,0.082056,6.0522,5.1945,3.0525,1.8459,0.120041,10.7886,8.7296,7.1231,0.084859,13.8244,13.0117,-0.0841,1,-0.71,-0.475,-0.649,-0.4182,-7.11,17,4.45,-3.26,2.56,0.2947,-0.7220,0,0,5,1
16,17,130,9,0.437883,1300,3,11255,3679.0,2969.0,129.74,130.00,14,0.9292,0.8974,1452127,0.8537,0.8468,0.7569,0.9211,0.9459,0.9211,2.207205e+09,105,77,1433250.0,105.0,105.0,138.75,120.19,105.0,0.9459,0,0,0,54,0,0,0.80340,0.83212,0.813783,...,60.86,55,2047.45,2269.90,227675.97,6.0002,-0.7169,25.9252,41.2324,36.7683,32.9873,0.141878,27.6148,24.7630,17.2124,12.1137,0.097355,39.3422,34.2219,29.9280,0.135889,44.2577,42.0351,-0.0541,1,-0.14,-0.220,-0.136,-0.5319,-28.72,54,19.33,-14.17,10.84,0.2740,0.7246,0,0,5,16
26,27,125,9,0.197708,1800,4,14694,2040.0,167.0,129.86,126.31,14,0.6613,0.6613,0,0.6667,0.6613,0.4936,0.6667,0.7387,0.7321,1.431862e+09,82,40,934636.0,82.0,82.0,82.00,82.00,82.0,0.8454,0,0,0,27,0,0,0.72065,0.67517,0.728470,...,49.37,28,728.08,2137.69,55788.79,4.7848,-0.7712,8.1536,18.2000,15.2198,12.8389,0.098816,8.8630,8.0841,4.6855,2.7847,0.119511,17.5564,14.4526,12.0148,0.102401,21.3140,19.8505,-0.2613,1,-0.39,-0.535,-0.421,-0.5363,-14.48,27,10.03,-7.64,6.06,0.2896,0.6349,0,0,5,19
50,51,122,9,0.451065,2200,4,8563,631.0,2547.0,125.07,122.61,13,0.8684,0.8049,9383293,0.8115,0.7615,0.6035,0.8462,0.9252,0.8609,3.321990e+09,99,66,1254528.0,99.0,99.0,138.55,116.52,99.0,0.9429,0,0,0,4,0,0,0.76075,0.76535,0.756900,...,-5.30,5,26.29,-112.97,536.90,2.0333,-0.9940,1.3150,2.9326,2.5220,2.1752,0.045635,1.6545,1.2953,0.7422,0.4272,0.038160,2.7722,2.3129,1.9323,0.031505,3.1618,2.9246,-0.0748,1,-0.67,-0.655,-0.673,-0.6150,-2.46,4,1.58,-1.04,0.72,0.1295,1.1789,0,0,5,4
64,65,153,10,0.829348,3800,4,78149,105.0,2724.0,142.82,150.53,15,0.7857,0.7586,542244,0.7521,0.7521,0.5375,0.7652,0.7928,0.7719,3.461878e+09,88,38,983488.0,88.0,88.0,134.95,108.27,88.0,0.8713,0,0,0,4,0,0,0.75605,0.75689,0.757575,...,12.23,5,289.04,2529.51,47273.70,7.1990,0.6675,1.3189,2.4970,1.9734,1.5599,0.010518,1.5314,1.3818,0.8160,0.4834,0.033838,2.4643,1.9350,1.5198,0.012798,2.8685,2.5964,-0.2072,1,-0.09,-0.355,-0.141,-0.4850,-1.94,4,1.15,-0.72,0.45,0.2286,0.9470,0,0,5,11


- Here we can see, the loss column in Loss daraframe contains different values in between(0-100).<br>
- As we want to do a binary classification to predict if the loan will be a defalt or not so, we don't need this loss percentage rather we need a indicator here to indicate that it's a default loan.
- As 0 indecates no loss(not default) so we can pick 1 to indicate (default)

In [8]:
# replacing loss column with 1 to indecate default loss

Loss['loss'] = Loss['loss'].apply(lambda x: 1)
Loss.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,...,f740,f741,f742,f743,f744,f745,f746,f747,f748,f749,f750,f751,f752,f753,f754,f755,f756,f757,f758,f759,f760,f761,f762,f763,f764,f765,f766,f767,f768,f769,f770,f771,f772,f773,f774,f775,f776,f777,f778,loss
7,8,128,9,0.385778,2900,4,79244,6597.0,3592.0,127.19,127.89,6,0.8596,0.7967,17175731,0.8033,0.7538,0.5868,0.8376,0.9159,0.8522,3.127331e+09,98,75,1219708.0,98.0,98.0,120.79,108.46,98.0,0.9899,0,0,0,17,0,0,0.72690,0.73842,0.739165,...,21.49,18,177.11,361.87,5280.10,2.9007,-0.7834,5.5179,11.5597,9.6605,8.1316,0.082056,6.0522,5.1945,3.0525,1.8459,0.120041,10.7886,8.7296,7.1231,0.084859,13.8244,13.0117,-0.0841,1,-0.71,-0.475,-0.649,-0.4182,-7.11,17,4.45,-3.26,2.56,0.2947,-0.7220,0,0,5,1
16,17,130,9,0.437883,1300,3,11255,3679.0,2969.0,129.74,130.00,14,0.9292,0.8974,1452127,0.8537,0.8468,0.7569,0.9211,0.9459,0.9211,2.207205e+09,105,77,1433250.0,105.0,105.0,138.75,120.19,105.0,0.9459,0,0,0,54,0,0,0.80340,0.83212,0.813783,...,60.86,55,2047.45,2269.90,227675.97,6.0002,-0.7169,25.9252,41.2324,36.7683,32.9873,0.141878,27.6148,24.7630,17.2124,12.1137,0.097355,39.3422,34.2219,29.9280,0.135889,44.2577,42.0351,-0.0541,1,-0.14,-0.220,-0.136,-0.5319,-28.72,54,19.33,-14.17,10.84,0.2740,0.7246,0,0,5,1
26,27,125,9,0.197708,1800,4,14694,2040.0,167.0,129.86,126.31,14,0.6613,0.6613,0,0.6667,0.6613,0.4936,0.6667,0.7387,0.7321,1.431862e+09,82,40,934636.0,82.0,82.0,82.00,82.00,82.0,0.8454,0,0,0,27,0,0,0.72065,0.67517,0.728470,...,49.37,28,728.08,2137.69,55788.79,4.7848,-0.7712,8.1536,18.2000,15.2198,12.8389,0.098816,8.8630,8.0841,4.6855,2.7847,0.119511,17.5564,14.4526,12.0148,0.102401,21.3140,19.8505,-0.2613,1,-0.39,-0.535,-0.421,-0.5363,-14.48,27,10.03,-7.64,6.06,0.2896,0.6349,0,0,5,1
50,51,122,9,0.451065,2200,4,8563,631.0,2547.0,125.07,122.61,13,0.8684,0.8049,9383293,0.8115,0.7615,0.6035,0.8462,0.9252,0.8609,3.321990e+09,99,66,1254528.0,99.0,99.0,138.55,116.52,99.0,0.9429,0,0,0,4,0,0,0.76075,0.76535,0.756900,...,-5.30,5,26.29,-112.97,536.90,2.0333,-0.9940,1.3150,2.9326,2.5220,2.1752,0.045635,1.6545,1.2953,0.7422,0.4272,0.038160,2.7722,2.3129,1.9323,0.031505,3.1618,2.9246,-0.0748,1,-0.67,-0.655,-0.673,-0.6150,-2.46,4,1.58,-1.04,0.72,0.1295,1.1789,0,0,5,1
64,65,153,10,0.829348,3800,4,78149,105.0,2724.0,142.82,150.53,15,0.7857,0.7586,542244,0.7521,0.7521,0.5375,0.7652,0.7928,0.7719,3.461878e+09,88,38,983488.0,88.0,88.0,134.95,108.27,88.0,0.8713,0,0,0,4,0,0,0.75605,0.75689,0.757575,...,12.23,5,289.04,2529.51,47273.70,7.1990,0.6675,1.3189,2.4970,1.9734,1.5599,0.010518,1.5314,1.3818,0.8160,0.4834,0.033838,2.4643,1.9350,1.5198,0.012798,2.8685,2.5964,-0.2072,1,-0.09,-0.355,-0.141,-0.4850,-1.94,4,1.15,-0.72,0.45,0.2286,0.9470,0,0,5,1


In [9]:
# appending both default and non-default loan information into a single dataframe.

data =  NoLoss.append(Loss)
data = data.sample(frac=1)
print("Shape of Dataset: ", data.shape)

Shape of Dataset:  (90000, 771)


## Dealing with missing values

In [10]:
# # Missing Values

# Function to calculate missing values by column
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [11]:
missing_values_table(data).shape


Your selected dataframe has 771 columns.
There are 525 columns that have missing values.


(525, 2)

![](http://)* filling all numeraical(column) missing value with mean value of corresponding colums'

In [12]:
data.fillna(data.mean(), inplace=True)


In [13]:
missing_values_table(data).shape


Your selected dataframe has 771 columns.
There are 12 columns that have missing values.


(12, 2)

- dropping rest of the missing value rows.

In [14]:
data.dropna(inplace=True)
missing_values_table(data)

Your selected dataframe has 771 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


- Splitting Data into train & test dataset.
- taking  50% of total data as test datser.

In [15]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size = 0.5)

- id is a redundant feature here
- loss is target feature.
So, taking all features except id and target for training perpous

In [16]:
features = [c for c in train_data.columns if c not in ['id', 'loss']]
target = train_data['loss']
tar = target
testingTarget = test_data['loss']

- as id is redundant and loss is the target column so dropping them from training feature list

In [17]:
train_data = train_data.drop(["id", "loss" ], axis=1)
test_data = test_data.drop(["id", "loss"], axis=1)


## Scaling all numeric train and test data features.

In [18]:
%%time
train_data = pd.DataFrame(scale(train_data.values), columns=train_data.columns, index=train_data.index)
test_data = pd.DataFrame(scale(test_data.values), columns=test_data.columns, index=test_data.index)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:180: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:180: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Nu

CPU times: user 5.8 s, sys: 2.6 s, total: 8.4 s
Wall time: 8.4 s


### to predict the loan default using light gradient boosting algorithm.

In [19]:
# parameters for lgbm
params = {}
params['bagging_freq'] = 5 #reducing it as smaller freq & frac reduce overfitting 
params['bagging_fraction'] = 0.0331
params['random_state'] = 42
params['learning_rate'] = 0.0123
params['boost_from_average'] = False
params['boosting_type'] = 'gbdt'
params['feature_fraction'] = 0.045
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['min_data_in_leaf'] = 80
params['num_leaves'] = 13
params['num_threads'] = 8
params['tree_learner'] = 'serial'
params['max_depth'] = -1
params['min_sum_hessian_in_leaf'] = 10.0
params['verbosity'] =  1
params['bagging_seed'] = 42
params['seed'] = 42

### using kfold cross validation

In [20]:
num_folds = 10
folds = StratifiedKFold(n_splits=num_folds,shuffle=True, random_state=42)
oof = np.zeros(len(train_data))
getVal = np.zeros(len(train_data))
predictions = np.zeros(len(tar))
print(predictions.shape)
feature_importance_df = pd.DataFrame()

(44031,)


- traing the model and predicting.
- taking the best iteration for prediction

In [21]:
%%time
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_data.values, target.values)):
    
    X_train, y_train = train_data.iloc[trn_idx][features], target.iloc[trn_idx]
    X_valid, y_valid = train_data.iloc[val_idx][features], target.iloc[val_idx]
    
    X_tr, y_tr = X_train.values, y_train.values
    X_tr = pd.DataFrame(X_tr)
    
    print("Fold idx:{}".format(fold_ + 1))
    trn_data = lgb.Dataset(X_tr, label=y_tr)
    val_data = lgb.Dataset(X_valid, label=y_valid)
    
    clf = lgb.train(params, trn_data, 10000, valid_sets = [trn_data, val_data], verbose_eval=500, early_stopping_rounds = 400)
   
    predictions += clf.predict(test_data[features], num_iteration=clf.best_iteration) / folds.n_splits

Fold idx:1
Training until validation scores don't improve for 400 rounds.
[500]	training's binary_logloss: 0.286571	valid_1's binary_logloss: 0.28927
[1000]	training's binary_logloss: 0.280136	valid_1's binary_logloss: 0.289399
Early stopping, best iteration is:
[622]	training's binary_logloss: 0.284786	valid_1's binary_logloss: 0.288387
Fold idx:2
Training until validation scores don't improve for 400 rounds.
[500]	training's binary_logloss: 0.286766	valid_1's binary_logloss: 0.286337
[1000]	training's binary_logloss: 0.280728	valid_1's binary_logloss: 0.285758
Early stopping, best iteration is:
[835]	training's binary_logloss: 0.282327	valid_1's binary_logloss: 0.285199
Fold idx:3
Training until validation scores don't improve for 400 rounds.
[500]	training's binary_logloss: 0.28597	valid_1's binary_logloss: 0.292896
[1000]	training's binary_logloss: 0.279553	valid_1's binary_logloss: 0.293079
Early stopping, best iteration is:
[828]	training's binary_logloss: 0.281201	valid_1's bina

- converting the prediction values into binary format

In [22]:
for i in range(0,len(predictions)):
    if predictions[i] <=0.5:
        predictions[i] = 0
    else: 
        predictions[i] = 1

In [23]:
from sklearn.metrics import accuracy_score
Accuracy = accuracy_score(testingTarget, predictions)

In [24]:
print("Accuracy: ", Accuracy)

Accuracy:  0.908110195089823
